# Question 1

Suppose that we would like to know how much families in the US are spending on recreation annually. We've estimated the following model:

𝑒𝑥𝑝𝑒𝑛𝑑𝑖𝑡𝑢𝑟𝑒 = 873 + 0.0012𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒 + 0.00002𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒**2 − 223.57ℎ𝑎𝑣𝑒_𝑘𝑖𝑑𝑠
<br>
<br>
expenditure is the annual spending on recreation in US dollars, annual_income is the annual income in US dollars, and have_kids is a dummy variable indicating the families with children. Interpret the estimated coefficients. What additional statistics should be given in order to make sure that your interpretations make sense statistically. Write up your answer.

1. Famlilies who have kids end up spending \$223.57 less than families who don't have kids
2. As a family's annual income increases, their recreation expenditure increases exponentially. The amount of increase is dependent upon the starting annual income.
3. It is difficult to explain the meaning of the constant term when there are this many explanatory features. It is safe to represent the constant term as the bias in our model.
4. The p-values for the coefficients terms should also be given. This will have the statistical signficance for each coefficient. If a p-value for a coefficient is less than .05, we can conclude the coefficient is effectively 0. 

# Question 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import scipy.stats as stats
%matplotlib inline

In [3]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

df = pd.read_sql_query('select * from weatherinszeged',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [4]:
df.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [5]:
df['temp_diff'] = df['apparenttemperature'] - df['temperature']

explanatory_var = ['humidity', 'windspeed']

# Y is our target variable
Y = df['temp_diff']

# X is our model features
X = df[explanatory_var]

In [6]:
import statsmodels.api as sm

X = sm.add_constant(X)

results = sm.OLS(Y,X).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              temp_diff   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Sat, 04 Jan 2020   Prob (F-statistic):               0.00
Time:                        11:11:47   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.0

//anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


temp_diff = 2.43 - 3.03 (humidity) - 0.12 (windspeed)

### Summary

1. The signs of the coefficients do make sense. An increase in windspeed results in a drop in temperature difference, thus making it feel colder. An increase in humidity also results in a drop in temperature difference. 

2. All the of the coefficients have p_values of 0, which indicates they are statistically significant.

3. An increase in 10 km/hr of windspeed results in a drop of 1.2 degrees temperature difference.

4. an increase in 10% humidity results in a drop of 0.303 degrees temperature difference.

In [7]:
df['humidity_windspeed_interaction'] = df['humidity']*df['windspeed']

X = df[explanatory_var + ['humidity_windspeed_interaction']]
Y = df['temp_diff']

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              temp_diff   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Sat, 04 Jan 2020   Prob (F-statistic):               0.00
Time:                        11:11:47   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const       

Temperature difference = .084 + .18 (humidity) + .10 (windspeed) - .30 (humidity) (windspeed)

### Summary

1. All of the coefficients are statistically significant.

2. The signs for windspeed and humidity coefficients have changed. 

3. For every 10 km/hr increase in windspeed the temperature difffernce changes by 1 - 3*(humidity)

4. For every 10% increase in humidity the temperature difffernce changes by .018 - .03*(windspeed)

# Question 3

In [8]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [9]:
df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


In [10]:
# List of continuous variables for our model
features = ['overallqual', 'grlivarea', 'garagecars', 'firstflrsf', 'fullbath', 'yearbuilt', 'yearremodadd']

# New Data frame with only our model features
model_df = pd.DataFrame()
for col in features:
    model_df[col] = df[col]


In [11]:
# adding the categorical features and salesprice to our model
model_df = pd.concat([model_df, pd.get_dummies(df[['neighborhood', 'exterqual', 'kitchenqual']], drop_first=True), df['saleprice']], axis=1)

In [12]:
# X is our features
X = model_df[['overallqual', 'grlivarea', 'garagecars', 'firstflrsf', 'fullbath',
       'yearbuilt', 'yearremodadd', 'neighborhood_Blueste',
       'neighborhood_BrDale', 'neighborhood_BrkSide', 'neighborhood_ClearCr',
       'neighborhood_CollgCr', 'neighborhood_Crawfor', 'neighborhood_Edwards',
       'neighborhood_Gilbert', 'neighborhood_IDOTRR', 'neighborhood_MeadowV',
       'neighborhood_Mitchel', 'neighborhood_NAmes', 'neighborhood_NPkVill',
       'neighborhood_NWAmes', 'neighborhood_NoRidge', 'neighborhood_NridgHt',
       'neighborhood_OldTown', 'neighborhood_SWISU', 'neighborhood_Sawyer',
       'neighborhood_SawyerW', 'neighborhood_Somerst', 'neighborhood_StoneBr',
       'neighborhood_Timber', 'neighborhood_Veenker', 'exterqual_Fa',
       'exterqual_Gd', 'exterqual_TA', 'kitchenqual_Fa', 'kitchenqual_Gd',
       'kitchenqual_TA']]

# Y is our target
Y = model_df['saleprice']

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       0.830
Model:                            OLS   Adj. R-squared:                  0.825
Method:                 Least Squares   F-statistic:                     187.3
Date:                Sat, 04 Jan 2020   Prob (F-statistic):               0.00
Time:                        11:11:49   Log-Likelihood:                -17252.
No. Observations:                1460   AIC:                         3.458e+04
Df Residuals:                    1422   BIC:                         3.478e+04
Df Model:                          37                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                -7.583e+05 

The features that are not statistically significant are fullbath, neighborhood_Blueste, neighborhood_BrDale, neighborhood_BrkSide, neighborhood_Edwards, neighborhood_Gilbert, neighborhood_IDOTRR, neighborhood_MeadowV, neighborhood_Mitchel, neighborhood_NAmes, neighborhood_NPkVill, neighborhood_NWAmes, neighborhood_OldTown, neighborhood_SWISU, neighborhood_Sawyer, and neighborhood_SawyerW.

In [13]:
# X is our features with the insignificant features excluded
X = model_df[['overallqual', 'grlivarea', 'garagecars', 'firstflrsf', 'yearbuilt', 'yearremodadd', 'neighborhood_ClearCr',
       'neighborhood_CollgCr', 'neighborhood_Crawfor', 'neighborhood_NoRidge', 'neighborhood_NridgHt', 'neighborhood_Somerst', 'neighborhood_StoneBr',
       'neighborhood_Timber', 'neighborhood_Veenker', 'exterqual_Fa', 'exterqual_Gd', 'exterqual_TA', 'kitchenqual_Fa', 'kitchenqual_Gd',
       'kitchenqual_TA']]

# Y is our target
Y = model_df['saleprice']

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       0.825
Model:                            OLS   Adj. R-squared:                  0.823
Method:                 Least Squares   F-statistic:                     323.9
Date:                Sat, 04 Jan 2020   Prob (F-statistic):               0.00
Time:                        11:11:49   Log-Likelihood:                -17270.
No. Observations:                1460   AIC:                         3.458e+04
Df Residuals:                    1438   BIC:                         3.470e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                -8.647e+05 

1. overallqual - A 1 point increase in overallqual results in \$1,259 increase in saleprice.
2. grlivarea - Each additional square foot in grlivarea results in \$41.87 increase in saleprice.
3. garagecars - A 1 point increase in garagecars results in \$1,196 increase in saleprice.
4. firstflrsf - Each additional square foot in firstflrsf results in \$21.57 increase in saleprice.
5. yearbuilt - For every year newer a house is results in a \$267.42 increase in saleprice.
6. yearremodadd - For every year newer a house is remodeled results in a \$196.44 increase in saleprice.
7. neighborhood_ClearCr - A house located in ClearCr results in a \$3,157 increase in saleprice.
8. neighborhood_CollgCr - A house located in CollgCr results in a \$8,951 increase in saleprice.
9. neighborhood_Crawfor - A house located in Crawfor results in a \$3,443 increase in saleprice.
10. neighborhood_NoRidge - A house located in NoRidge results in a \$6,910 increase in saleprice.
11. neighborhood_NridgHt - A house located in NridgHt results in a \$3,939 increase in saleprice.
12. neighborhood_Somerst - A house located in Somerst results in a \$1,230 increase in saleprice.
13. neighborhood_StoneBr - A house located in StoneBr results in a \$5,572 increase in saleprice.
14. neighborhood_Timber - A house located in Timber results in a \$2,271 increase in saleprice.
15. neighborhood_Veenker - A house located in Veenker results in a \$4,208 increase in saleprice.
16. exterqual_Fa - An exterior qulaity of Fa results in a \$3,638 decrease in saleprice.
17. exterqual_Gd - An exterior qulaity of Gd results in a \$3,491 decrease in saleprice.
18. exterqual_TA - An exterior qulaity of Ta results in a \$3,370 decrease in saleprice.
19. kitchenqual_Fa - A kitchen quality of Fa results in a \$3,811 decrease in saleprice.
20. kitchenqual_Gd - A kitchen qulaity of Gd results in a \$3,411 decrease in saleprice.
21. kitchenqual_TA - A kitchen qulaity of Ta results in a \$3,894 decrease in saleprice.

All the results seem reasonable. I believe grlivarea is the most prominent feature. A house with 1000 additional square feet will sell for \$41,870 more.